# Training Pipeline

Complete pipeline for data preparation and LSTM model training for all decision points.


In [5]:
import sys
import os
import warnings

sys.path.insert(0, os.path.dirname(os.getcwd()))

import numpy as np
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split

from pm4py.objects.bpmn.importer import importer as bpmn_importer
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.conversion.log import converter as log_converter

from preprocessing import DecisionPointExtractor, TrainingDataGenerator
from models import SequenceEncoder, ContextEncoder, LSTMPredictor
from storage import ModelPersistence

pd.options.mode.chained_assignment = None
warnings.filterwarnings("ignore")


## 1. Configuration


In [6]:
BPMN_PATH = os.path.join(os.getcwd(), "..", "..", "..", "process_model", "loan_application.bpmn")
XES_PATH = os.path.join(os.getcwd(), "..", "..", "..", "Dataset", "BPI Challenge 2017.xes")
MODELS_DIR = os.path.join(os.getcwd(), "..", "models_lstm_new")
DATA_DIR = os.path.join(os.getcwd(), "..", "data", "processed")

CONTEXT_KEYS = ["case:LoanGoal", "case:ApplicationType", "case:RequestedAmount"]
MAX_HISTORY = 15
MIN_SEQ_COUNT = 20
MIN_CLASS_COUNT = 10
RANDOM_STATE = 42


## 2. Load Data


In [7]:
print(f"Loading BPMN: {BPMN_PATH}")
bpmn_model = bpmn_importer.apply(BPMN_PATH)

print(f"Loading event log: {XES_PATH}")
event_log = xes_importer.apply(XES_PATH)
df_log = log_converter.apply(event_log, variant=log_converter.Variants.TO_DATA_FRAME)

print(f"Loaded {len(df_log)} events from {df_log['case:concept:name'].nunique()} cases")


Loading BPMN: d:\Repos\process-simulation-engine\Next-Activity-Prediction\advanced\notebooks\..\..\..\process_model\loan_application.bpmn
Loading event log: d:\Repos\process-simulation-engine\Next-Activity-Prediction\advanced\notebooks\..\..\..\Dataset\BPI Challenge 2017.xes


parsing log, completed traces :: 100%|██████████| 31509/31509 [00:36<00:00, 854.72it/s] 


Loaded 1202267 events from 31509 cases


## 3. Extract Decision Points


In [8]:
extractor = DecisionPointExtractor(bpmn_model)
dp_map = extractor.extract()

print(f"Found {len(dp_map)} decision points")
for dp, config in list(dp_map.items())[:3]:
    print(f"  {dp}: {len(config['incoming'])} incoming, {len(config['outgoing'])} outgoing")


Found 49 decision points
  DP 1: 1 incoming, 5 outgoing
  DP 2: 2 incoming, 1 outgoing
  DP 3: 1 incoming, 5 outgoing


## 4. Prepare Event Log


In [9]:
columns = [
    "case:concept:name", "concept:name", "org:resource",
    "time:timestamp", "case:LoanGoal", "case:ApplicationType", "case:RequestedAmount"
]

df = df_log[columns].copy()
df["time:timestamp"] = pd.to_datetime(df["time:timestamp"])
df = df.sort_values(["case:concept:name", "time:timestamp"]).reset_index(drop=True)

end_events = df.sort_values("time:timestamp").groupby("case:concept:name").tail(1).copy()
end_events["time:timestamp"] += pd.Timedelta(seconds=1)
end_events["concept:name"] = "End"

df = pd.concat([df, end_events], ignore_index=True)
df = df.sort_values(["case:concept:name", "time:timestamp"]).reset_index(drop=True)

print(f"Prepared {len(df)} events with End markers")


Prepared 1233776 events with End markers


## 5. Generate Training Data


In [14]:
dp_datasets = {}

for dp_name, dp_config in dp_map.items():
    generator = TrainingDataGenerator(
        df, 
        {dp_name: dp_config},
        max_history=MAX_HISTORY,
        min_seq_count=MIN_SEQ_COUNT,
        min_class_count=MIN_CLASS_COUNT
    )
    result = generator.generate()
    
    if dp_name in result:
        dp_datasets[dp_name] = result[dp_name]
        print(f"{dp_name}: {len(result[dp_name])} samples, {result[dp_name]['label'].nunique()} classes")
    else:
        print(f"{dp_name}: no valid samples")

print(f"\nGenerated datasets for {len(dp_datasets)} decision points")


DP 1: 31509 samples, 3 classes
DP 2: no valid samples
DP 3: 47079 samples, 2 classes
DP 4: 31441 samples, 2 classes
DP 5: 31441 samples, 2 classes
DP 6: no valid samples
DP 7: 113969 samples, 4 classes
DP 8: 145410 samples, 4 classes
DP 9: 179311 samples, 4 classes
DP 10: 179311 samples, 4 classes
DP 11: 179256 samples, 3 classes
DP 12: 179256 samples, 3 classes
DP 13: 178967 samples, 2 classes
DP 14: 36213 samples, 2 classes
DP 15: 36213 samples, 2 classes
DP 16: 33084 samples, 6 classes
DP 17: 33084 samples, 6 classes
DP 18: no valid samples
DP 19: 179857 samples, 7 classes
DP 20: 179857 samples, 7 classes
DP 21: 179786 samples, 6 classes
DP 22: 179786 samples, 6 classes
DP 23: 70745 samples, 7 classes
DP 24: no valid samples
DP 25: 5469 samples, 5 classes
DP 26: 73711 samples, 7 classes
DP 27: 426576 samples, 8 classes
DP 28: 426576 samples, 8 classes
DP 29: 426576 samples, 8 classes
DP 30: 426576 samples, 8 classes
DP 31: 185464 samples, 8 classes
DP 32: 426576 samples, 8 classes
D

## 6. Split Data


In [ ]:
splits = {}

for dp, data in dp_datasets.items():
    data = data.drop(columns=["sequence_timestamps"], errors="ignore")
    
    temp, holdout = train_test_split(
        data, test_size=0.1, random_state=RANDOM_STATE,
        stratify=data["label"] if data["label"].nunique() > 1 else None
    )
    train, test = train_test_split(
        temp, test_size=2/9, random_state=RANDOM_STATE,
        stratify=temp["label"] if temp["label"].nunique() > 1 else None
    )
    
    splits[dp] = {
        "train": train.reset_index(drop=True),
        "test": test.reset_index(drop=True),
        "holdout": holdout.reset_index(drop=True)
    }
    print(f"{dp}: train={len(train)}, test={len(test)}, holdout={len(holdout)}")

os.makedirs(DATA_DIR, exist_ok=True)
joblib.dump(splits, os.path.join(DATA_DIR, "dp_split_datasets_full_simple.joblib"))
print(f"\nSaved splits to {DATA_DIR}")


DP 1: train=22056, test=6302, holdout=3151
DP 3: train=32955, test=9416, holdout=4708
DP 4: train=22008, test=6288, holdout=3145
DP 5: train=22008, test=6288, holdout=3145
DP 7: train=79778, test=22794, holdout=11397
DP 8: train=101787, test=29082, holdout=14541
DP 9: train=125517, test=35862, holdout=17932
DP 10: train=125517, test=35862, holdout=17932
DP 11: train=125478, test=35852, holdout=17926
DP 12: train=125478, test=35852, holdout=17926
DP 13: train=125276, test=35794, holdout=17897
DP 14: train=25348, test=7243, holdout=3622
DP 15: train=25348, test=7243, holdout=3622
DP 16: train=23158, test=6617, holdout=3309
DP 17: train=23158, test=6617, holdout=3309
DP 19: train=125899, test=35972, holdout=17986
DP 20: train=125899, test=35972, holdout=17986
DP 21: train=125849, test=35958, holdout=17979
DP 22: train=125849, test=35958, holdout=17979
DP 23: train=49521, test=14149, holdout=7075
DP 25: train=3828, test=1094, holdout=547
DP 26: train=51597, test=14742, holdout=7372
DP 27: 

## 7. Train Models


In [12]:
trained_models = {}

for dp, split in splits.items():
    print(f"\n=== Training {dp} ===")
    
    df_pool = pd.concat([split["train"], split["test"]], ignore_index=True)
    df_all = pd.concat([df_pool, split["holdout"]], ignore_index=True)
    n_pool = len(df_pool)
    
    seq_encoder = SequenceEncoder()
    X_acts, X_durs, X_res, y = seq_encoder.fit_transform(df_all)
    seq_encoder.add_unknown_token()
    
    ctx_encoder = ContextEncoder(CONTEXT_KEYS)
    X_ctx = ctx_encoder.fit_transform(df_all)
    
    X_acts_train, X_acts_test = X_acts[:n_pool], X_acts[n_pool:]
    X_durs_train, X_durs_test = X_durs[:n_pool], X_durs[n_pool:]
    X_res_train, X_res_test = X_res[:n_pool], X_res[n_pool:]
    X_ctx_train, X_ctx_test = X_ctx[:n_pool], X_ctx[n_pool:]
    y_train, y_test = y[:n_pool], y[n_pool:]
    
    predictor = LSTMPredictor(
        num_activities=seq_encoder.num_activities,
        num_resources=seq_encoder.num_resources,
        context_dim=ctx_encoder.dim,
        max_seq_len=seq_encoder.max_seq_len,
        num_classes=seq_encoder.num_classes
    )
    predictor.build()
    predictor.train(
        [X_acts_train, X_durs_train, X_res_train, X_ctx_train],
        y_train
    )
    
    trained_models[dp] = {
        "model": predictor.model,
        "activity_encoder": seq_encoder.activity_encoder,
        "resource_encoder": seq_encoder.resource_encoder,
        "label_encoder": seq_encoder.label_encoder,
        "context_encoders": ctx_encoder.encoders,
        "context_keys": CONTEXT_KEYS,
        "max_seq_len": seq_encoder.max_seq_len
    }

print(f"\nTrained {len(trained_models)} models")



=== Training DP 13 ===
Epoch 1/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.9382 - loss: 0.1846 - val_accuracy: 0.9540 - val_loss: 0.1711
Epoch 2/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.9620 - loss: 0.1457 - val_accuracy: 0.9579 - val_loss: 0.1617
Epoch 3/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.9635 - loss: 0.1413 - val_accuracy: 0.9568 - val_loss: 0.1677
Epoch 4/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.9641 - loss: 0.1405 - val_accuracy: 0.9590 - val_loss: 0.1623
Epoch 5/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9642 - loss: 0.1384 - val_accuracy: 0.9601 - val_loss: 0.1620
Epoch 6/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.9643 - loss: 0.1378 - val_accuracy: 0.9576 - val_loss: 0.1624
Epoch 7/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9644 - loss: 0.1369 - val_accuracy: 0.9590 - val_loss: 0.1592
Epoch 8/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9647 -

## 8. Save Models


In [13]:
ModelPersistence.save_all(trained_models, MODELS_DIR)
print(f"Saved all models to {MODELS_DIR}")


Saved all models to d:\Repos\process-simulation-engine\Next-Activity-Prediction\advanced\notebooks\..\models_lstm_new
